In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import requests
import json
import pprint
import time

import pandas as pd

# fix ssl certificate (needed for MacOS sometimes)
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

CAP_FRIENDLY_BASE_URL = 'https://www.capfriendly.com'

/Users/davidshaw/Projects/minWage/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
pages = []
total_num_scraped = 0
seasons = range(2005,2025)
for season in seasons:
    print(f'#####################\nScraping Season: {season}')
    
    # reset pagination tokens
    pag_tradeId = 0
    pag_date = ''
    
    result_count = 100 # set to large amount so we can enter the while loop
    while result_count >= 24:
        url = f'{CAP_FRIENDLY_BASE_URL}/ajax2/trades?season={season}&offset-date={pag_date}&offset-trade_id={pag_tradeId}'
        trade_json = requests.get(url)

        r = requests.get(url)
        d = r.json()
        d['season'] = season
        pages.append(d)

        pag_tradeId = d['data']['pagination']['tradeId']
        pag_date = d['data']['pagination']['date']

        result_count = d['data']['result_count']
        total_num_scraped += result_count
            

        print(d['data']['pagination'], end='; ')
        print(f"NumResults: {d['data']['result_count']}")
        time.sleep(0.1)
        
print(f'Total Scraped: {total_num_scraped}')

#####################
Scraping Season: 2005
{'tradeId': 5315, 'date': '2004-07-06'}; NumResults: 3
#####################
Scraping Season: 2006
{'tradeId': 5244, 'date': '2006-06-23'}; NumResults: 24
{'tradeId': 5167, 'date': '2006-03-09'}; NumResults: 24
{'tradeId': 5130, 'date': '2006-01-23'}; NumResults: 24
{'tradeId': 5738, 'date': '2005-10-23'}; NumResults: 24
{'tradeId': 5186, 'date': '2005-08-02'}; NumResults: 24
{'tradeId': 4967, 'date': '2005-07-05'}; NumResults: 23
#####################
Scraping Season: 2007
{'tradeId': 5028, 'date': '2007-06-18'}; NumResults: 24
{'tradeId': 5032, 'date': '2007-02-27'}; NumResults: 24
{'tradeId': 5787, 'date': '2007-02-10'}; NumResults: 24
{'tradeId': 5089, 'date': '2006-11-27'}; NumResults: 24
{'tradeId': 4926, 'date': '2006-07-09'}; NumResults: 24
{'tradeId': 4973, 'date': '2006-07-02'}; NumResults: 3
#####################
Scraping Season: 2008
{'tradeId': 5695, 'date': '2008-06-20'}; NumResults: 24
{'tradeId': 5829, 'date': '2008-02-26'}; N

In [3]:
with open('tradesAsHTML_tilJuly2024.json', 'w') as f:
    json.dump(pages, f)